In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import re
from plotly.offline import iplot
from collections import defaultdict
import json

In [2]:
df = pd.read_csv('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/ED Events - 12.21.23.csv')
df['Arrived_Time'] = pd.to_datetime(df['Arrived_Time'])
df['Calculated_DateTime'] = pd.to_datetime(df['Calculated_DateTime'])
df = df.sort_values(by='Calculated_DateTime')

In [3]:
df = df[df['Calculated_DateTime'].dt.year>=2021]

In [15]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17304895 entries, 3024075 to 17277846
Data columns (total 31 columns):
 #   Column                                                                  Dtype         
---  ------                                                                  -----         
 0   PAT_ENC_CSN_ID                                                          int64         
 1   PAT_MRN_ID                                                              int64         
 2   PAT_ID                                                                  object        
 3   Ethnicity                                                               object        
 4   FirstRace                                                               object        
 5   MultiRacial                                                             int64         
 6   Sex                                                                     object        
 7   Arrived_Time                                  

In [4]:
df_admi = df[df['ED_Disposition']=='Admitted']
df_disch = df[df['ED_Disposition']=='Discharged']
df_all = pd.concat([df_admi, df_disch], axis=0, ignore_index=True)

In [14]:
(df['Patient_Age'] == 'Unknown').sum()

0

### Create a test raw db

In [6]:
r_subj = np.random.choice(df_all['PAT_ENC_CSN_ID'].unique(), 1000, replace=False)

In [10]:
df_random = df.loc[df['PAT_ENC_CSN_ID'].isin(r_subj)]

In [11]:
df_random.shape

(150964, 31)

In [12]:
df_random.to_csv('/work/InternalMedicine/s223850/ED-StaticDynamic/raw_data/ED Events - 12.21.23_random.csv')

In [ ]:
#======================================================

In [10]:
df[df['PAT_ENC_CSN_ID']==687744741]

,PAT_ENC_CSN_ID,PAT_MRN_ID,PAT_ID,Ethnicity,FirstRace,MultiRacial,Sex,Arrived_Time,ED_Disposition,Admitted_YN,...,Number of past appointments in last 60 days,Number of past inpatient admissions over ED visits in last three years,Type,EVENT_NAME,ED_Location_YN,Order_Status,Result_Flag,Primary_DX_Name,Primary_DX_First_ICD10,Primary_DX_ICD10


In [5]:
# Admissions without admission orders
group_admi = df_admi.groupby('PAT_ENC_CSN_ID')
pat_id_list = []
for pid, df_g in group_admi:
    mask_admi = df_g['Type'] == 'Order - Admission'
    if mask_admi.sum()==0:
        pat_id_list.append(pid)

In [27]:
# Discharges without discharge orders orders
group_disch = df_disch.groupby('PAT_ENC_CSN_ID')
pat_id_list_disc = []
for pid, df_g in group_disch:
    mask_disch = df_g['Type'] == 'Order - Discharge'
    if mask_disch.sum()==0:
        pat_id_list_disc.append(pid)

In [6]:
len(pat_id_list)/df_admi['PAT_ENC_CSN_ID'].nunique()

0.006760008750820389

In [31]:
len(pat_id_list_disc)/df_disch['PAT_ENC_CSN_ID'].nunique()

0.9971057134176567

In [26]:
pd.DataFrame(pat_id_list, columns=['PAT_ENC_CSN_ID']).to_csv('admit_patient_without_OrderAdmission.csv')

In [28]:
pd.DataFrame(pat_id_list_disc, columns=['PAT_ENC_CSN_ID']).to_csv('disch_patient_without_OrderDischarge.csv')

In [54]:
df[df['PAT_ENC_CSN_ID']==pat_id_list_disc[-2]].tail(60)

,PAT_ENC_CSN_ID,PAT_MRN_ID,PAT_ID,Ethnicity,FirstRace,MultiRacial,Sex,Arrived_Time,ED_Disposition,Admitted_YN,...,Number of past appointments in last 60 days,Number of past inpatient admissions over ED visits in last three years,Type,EVENT_NAME,ED_Location_YN,Order_Status,Result_Flag,Primary_DX_Name,Primary_DX_First_ICD10,Primary_DX_ICD10
882832,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,CALCIUM (ALBUMIN ADJUSTED),Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
1353039,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,BUN,Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
16304263,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,TOTAL PROTEIN,Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
1353038,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,ANION GAP,Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
16304262,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,GLOBULIN,Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
11577183,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,ALK PHOS,Yes,NaN,Low,RLQ abdominal pain,R10.31,R10.31
11577184,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,ALT,Yes,NaN,Low,RLQ abdominal pain,R10.31,R10.31
11577185,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,AST,Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
11577186,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,"BILIRUBIN, TOTAL",Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31
1481140,695159895,93148851,Z5607287,Non-Hispanic/Latino,Asian,0,Female,2023-12-20 19:48:28,Discharged,Not Admitted,...,0,0,Lab Order - Result,ALBUMIN,Yes,NaN,Normal,RLQ abdominal pain,R10.31,R10.31


In [5]:
df_all[f'EVENT_NAME_NORM'] = df_all['EVENT_NAME'].apply(lambda x: re.sub(r'[^a-zA-Z0-9_]', '', x)).str.lower()
df_all = df_all.sort_values(by='Calculated_DateTime')

In [6]:
def get_target_flags(raw_file_path, event_col='EVENT_NAME', grb_col='Admitted_YN', flag='consu', normalize_text=True, corr_threshold=0.7):

    df_raw = raw_file_path
    if normalize_text:
        # df_raw[f'{event_col}_NORM'] = df_raw[event_col].apply(lambda x: re.sub(r'[^a-zA-Z0-9_]', '', x)).str.lower()
        event_col = f'{event_col}_NORM'

    flag_df = df_raw.loc[df_raw[event_col].str.lower().str.contains(flag), [event_col, grb_col]]
    flag_grb = flag_df.groupby(grb_col).value_counts()

    data = pd.DataFrame(0, index=flag_df[event_col].unique(), columns=flag_df[grb_col].unique())
    for (flag, event), value in flag_grb.items():
        data.loc[event, flag] = value
    
    data['total'] = data.sum(axis=1)
    # TODO: This is a hardcoded value of the target params and need to be changed manually
    data['perc_pos'] = data['Admitted']/data['total']

    target_events = data.loc[(data['perc_pos']>=corr_threshold)|(data['perc_pos']<=(1-corr_threshold))].index
    return target_events, data


In [83]:
admi_flags, df_admi_flags  = get_target_flags(df_all, event_col='EVENT_NAME', grb_col='ED_Disposition', flag='consu', normalize_text=True, corr_threshold=0.7)

In [84]:
admi_flags_times = defaultdict(list) # in minutes
df_admi = df_all[df_all['ED_Disposition']=='Admitted']
group_admi = df_admi.groupby('PAT_ENC_CSN_ID')

for grb, df_grb in group_admi:
    admi_mask = df_grb['EVENT_NAME_NORM'].isin(admi_flags)
    if admi_mask.sum() == 0:
        continue
    mask_admission = df_grb['Type'] == 'Order - Admission'
    if mask_admission.sum() > 0:
        admi_time = df_grb.iloc[np.where(mask_admission)[0][0]]['Calculated_DateTime']
        for idx, row in df_grb[admi_mask].iterrows():
            admi_flags_times[row['EVENT_NAME_NORM']].append(('admission', grb, (admi_time-row['Calculated_DateTime']).total_seconds()/60))

In [85]:
flag_time_dict = defaultdict(list)
for flag, data_list in admi_flags_times.items():
    for d in data_list:
        flag_time_dict[flag].append(d[-1])

In [86]:
for flag, timelist in flag_time_dict.items():
    print(f'{flag} has min time: {np.min(timelist)}, median time: {np.median(timelist)}, average time {np.mean(timelist)}, std time {np.std(timelist, ddof=1)}, and max time: {np.max(timelist)} ..')

ippcconsulttoophthalmology has min time: -677.0, median time: 231.5, average time 253.26315789473685, std time 230.85810935363207, and max time: 1288.0 ..
consultorderedophthalmology has min time: -92.55, median time: 244.14166666666665, average time 282.4245575221239, std time 197.50653930358266, and max time: 1287.0 ..
consulttrackingstarted has min time: -1181.5833333333333, median time: 39.9, average time 111.56772399516649, std time 259.28983303712045, and max time: 9900.683333333332 ..
pharmacykineticsconsultvancomycin has min time: -1040.0, median time: 113.0, average time 128.87946555481852, std time 178.03412111954157, and max time: 1615.0 ..
ippcconsulttohematology has min time: -1151.0, median time: 45.0, average time 61.30362116991643, std time 188.74998706192963, and max time: 1764.0 ..
consultorderedhemoncattending has min time: -61.75, median time: 77.28333333333333, average time 124.0965721040189, std time 157.27162447372376, and max time: 1763.3333333333333 ..
ednuredc

/home2/s223850/.conda/envs/ED/lib/python3.10/site-packages/numpy/core/_methods.py:264: RuntimeWarning:

Degrees of freedom <= 0 for slice

/home2/s223850/.conda/envs/ED/lib/python3.10/site-packages/numpy/core/_methods.py:256: RuntimeWarning:

invalid value encountered in double_scalars



In [87]:
len(flag_time_dict)

225

In [89]:
len(list(filter(lambda x: np.median(x[1])>0,flag_time_dict.items())))

163

In [90]:
with open('consu_featurs.json', 'w') as f:
    json.dump(list(map(lambda x: x[0], list(filter(lambda x: np.median(x[1])>0,flag_time_dict.items())))),f)

In [160]:
t = list(map(lambda x: x[-1], admi_flags_times['hospitalistadmitsconsultcalled']))

In [161]:
np.argmin(t)

1996

In [162]:
admi_flags_times['hospitalistadmitsconsultcalled'][1996]

('admission', 655032721, -752.5)

In [163]:
pat_id_related = []
for x in admi_flags_times.items():
    for op, pid, t in x[1]:
        if pid==655032721:
            pat_id_related.append((x[0], op, pid, t))

In [164]:
pat_id_related

[('hospitalistadmitsconsultcalled', 'admission', 655032721, -752.5),
 ('antibioticsadministerediv', 'admission', 655032721, 7.716666666666667),
 ('antibioticsadministerediv', 'admission', 655032721, -7.466666666666667),
 ('antibioticsadministerediv', 'admission', 655032721, -41.6),
 ('antibioticsadministerediv', 'admission', 655032721, -88.11666666666666),
 ('antibioticsadministerediv', 'admission', 655032721, -424.71666666666664),
 ('antibioticsadministerediv', 'admission', 655032721, -729.75),
 ('antibioticsadministerediv', 'admission', 655032721, -729.8333333333334),
 ('antibioticsadministerediv', 'admission', 655032721, -889.6833333333333),
 ('potentialadmissionbedboardnotification', 'admission', 655032721, 0.0),
 ('admitstatuschange', 'admission', 655032721, -0.75),
 ('uhadmitorderplaced', 'admission', 655032721, -0.75),
 ('admissionordersreconciled', 'admission', 655032721, -62.266666666666666),
 ('admissionordersreconciled', 'admission', 655032721, -68.55),
 ('admitasoutpatient'

In [166]:
df_pat = df_all[df_all['PAT_ENC_CSN_ID']==655032721]

In [167]:
df_pat[['Calculated_DateTime', 'EVENT_NAME', 'Type', 'Order_Status', 'Result_Flag']]

,Calculated_DateTime,EVENT_NAME,Type,Order_Status,Result_Flag
1627889,2022-01-07 10:19:44,Emergency encounter created,Event,NaN,NaN
3623599,2022-01-07 10:20:04,Prev Admission Rec Generated,Event,NaN,NaN
6819359,2022-01-07 10:20:04,Admission FAM List Generated,Event,NaN,NaN
6819358,2022-01-07 10:20:04,Patient arrived in ED,Event,NaN,NaN
6378078,2022-01-07 10:20:04,Admission Med List Generated,Event,NaN,NaN
...,...,...,...,...,...
997639,2022-01-08 14:55:28,Pt rounding completed,Event,NaN,NaN
1784264,2022-01-08 14:55:28,Pt rounding timer started,Event,NaN,NaN
1784267,2022-01-08 15:13:26,Hospital bed ready,Event,NaN,NaN
3029725,2022-01-08 16:21:18,Bed Assigned,Event,NaN,NaN


In [170]:
admi_mask = df_pat['EVENT_NAME_NORM'].isin(admi_flags)
admi_mask.sum()

16

In [171]:
mask_admi = df_pat['Type'] == 'Order - Admission'
mask_admi.sum()

2

In [173]:
df_pat[mask_admi|admi_mask][['Type', 'EVENT_NAME', 'Calculated_DateTime']]

,Type,EVENT_NAME,Calculated_DateTime
8252312,Event,Hospitalist Admits Consult Ordered,2022-01-07 23:24:16
8252313,Event,Antibiotics Administered IV,2022-01-07 23:33:17
6614254,Order - Admission,POTENTIAL ADMISSION BED BOARD NOTIFICATION,2022-01-07 23:41:00
2872753,Event,Admit Status Change,2022-01-07 23:41:45
6647138,Event,UH Admit Order Placed,2022-01-07 23:41:45
6599820,Event,Antibiotics Administered IV,2022-01-07 23:48:28
6396697,Event,Antibiotics Administered IV,2022-01-08 00:22:36
6396699,Event,Admission Orders Reconciled,2022-01-08 00:43:16
6573966,Event,Admission Orders Reconciled,2022-01-08 00:49:33
774693,Order - Admission,ADMIT AS OUTPATIENT,2022-01-08 01:03:00


In [140]:
admi_time = df_pat.iloc[np.where(mask_admi)[0][0]]['Calculated_DateTime']
for idx, row in df_pat[admi_mask].iterrows():
    print(row['EVENT_NAME_NORM'], " : ", (admi_time-row['Calculated_DateTime']).total_seconds()/60)

admitasoutpatient  :  0.0
antibioticsadministerediv  :  -807.6833333333333
antibioticsadministerediv  :  -342.71666666666664
antibioticsadministerediv  :  -6.116666666666666
admitstatuschange  :  81.25
antibioticsadministerediv  :  -647.75
hospitalistadmitsconsultcalled  :  -670.5
antibioticsadministerediv  :  -647.8333333333334
antibioticsadministerediv  :  40.4
admissionordersreconciled  :  19.733333333333334
admissionordersreconciled  :  13.45
antibioticsadministerediv  :  74.53333333333333
potentialadmissionbedboardnotification  :  82.0
uhadmitorderplaced  :  81.25
hospitalistadmitsconsultordered  :  98.73333333333333
antibioticsadministerediv  :  89.71666666666667


In [97]:
mask_admission =df_pat['Type'] == 'Order - Admission'
mask_disch = df_pat['Type'] == 'Order - Discharge'

In [103]:
admi_time = df_pat.iloc[np.where(mask_disch)[0][0]]['Calculated_DateTime']
admi_time

Timestamp('2023-10-12 22:18:00')

In [104]:
df_pat[mask_disch]['Calculated_DateTime']

13506915   2023-10-12 22:18:00
Name: Calculated_DateTime, dtype: datetime64[ns]

In [82]:
df_pat[df_pat['EVENT_NAME_NORM']=='hospitalistadmitsconsultcalled']['Calculated_DateTime']

9130538   2023-10-12 18:09:15
Name: Calculated_DateTime, dtype: datetime64[ns]

In [79]:
df_pat['Type'].value_counts()

Event                 121
Lab Order - Result     70
Order - Lab            33
Order - Nursing        27
Order - Medication     23
Order - Imaging        13
Order - Consult         2
Order - Discharge       1
Name: Type, dtype: int64

In [85]:
(df_pat[df_pat['Type'] == 'Order - Discharge']['Calculated_DateTime'].iloc[0]-df_pat[df_pat['EVENT_NAME_NORM']=='hospitalistadmitsconsultcalled']['Calculated_DateTime'].iloc[0]).total_seconds()/60

248.75

# HOw many Type='Order - Admission', Type='Order - Discharge' per subject.
# How many different EVENT_NAME for each ORDER - Admission and Order - Discharge

In [11]:
groups = df_all.groupby('PAT_ENC_CSN_ID')
admi_dis_dict = {}
for g, df_g in groups:
    admi_mask = df_g['Type']=='Order - Admission'
    dish_mask = df_g['Type']=='Order - Discharge'
    admi_dis_dict[g] = (admi_mask.sum(), dish_mask.sum())
    

In [15]:
admi_orders = list(map(lambda x: x[1][0], admi_dis_dict.items()))

In [21]:
np.min(admi_orders), np.median(admi_orders), np.mean(admi_orders), np.max(admi_orders)

(0, 0.0, 0.7807883821260635, 10)

In [57]:
admi_orders_10 = list(filter(lambda x: x[1][0]==10, admi_dis_dict.items()))

In [36]:
len(admi_orders_1)

15915

In [38]:
len(admi_orders_2)

23979

In [42]:
len(admi_orders_3)

4789

In [44]:
len(admi_orders_5)

262

In [47]:
len(admi_orders_6)

72

In [50]:
len(admi_orders_7)

20

In [53]:
len(admi_orders_8)

7

In [56]:
len(admi_orders_9)

6

In [60]:
len(admi_orders_10), admi_orders_10

(1, [(685379149, (10, 0))])

In [63]:
df_all[(df_all['PAT_ENC_CSN_ID']==685379149)&(df_all['Type']=='Order - Admission')][['Calculated_DateTime', 'EVENT_NAME', 'Order_Status']]

,Calculated_DateTime,EVENT_NAME,Order_Status
5509324,2023-07-17 05:24:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Canceled
5509325,2023-07-17 05:29:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Canceled
5051725,2023-07-17 08:44:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Canceled
6600140,2023-07-17 09:50:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Canceled
6314670,2023-07-17 09:50:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Canceled
5500204,2023-07-17 13:28:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Canceled
6600144,2023-07-17 13:28:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Completed
6600145,2023-07-17 13:29:00,POTENTIAL ADMISSION BED BOARD NOTIFICATION,Completed
6600793,2023-07-17 13:56:00,ADMIT AS OBSERVATION,Completed
5253713,2023-07-17 13:57:00,ADMIT AS OBSERVATION,Completed


In [65]:
df_all[df_all['Type'] == 'Order - Admission']['EVENT_NAME'].value_counts()

POTENTIAL ADMISSION BED BOARD NOTIFICATION    49413
ADMIT AS INPATIENT                            20061
ADMIT AS OBSERVATION                          13913
ADMIT AS OUTPATIENT                            1073
ADMIT AS OUTPATIENT-IN-A-BED                     58
Name: EVENT_NAME, dtype: int64

In [66]:
df_all[df_all['Type'] == 'Order - Discharge']['EVENT_NAME'].value_counts()

DISCHARGE PATIENT            389
DISCHARGE EXPIRED PATIENT      2
Name: EVENT_NAME, dtype: int64